In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install spectral
!pip install -q -U tensorflow-addons
!pip install scikit-image --upgrade

In [ ]:
import spectral.io.envi as envi
from spectral import *
import numpy as np
from tqdm import tqdm

In [ ]:
import tensorflow_addons as tfa
from skimage import filters

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, UpSampling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate, Add, Activation, Multiply
from tensorflow.keras.models import Model
from tensorflow.python.keras.preprocessing import dataset_utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import relu
import numpy as np
from tifffile import imshow, imread

In [ ]:
img2 = envi.open('/content/drive/MyDrive/Diploma/data_new/F15_20160101_20161231.v4.stable_lights.shiftx1y1.avg_vis.hdr', '/content/drive/MyDrive/Diploma/data_new/F15_20160101_20161231.v4.stable_lights.shiftx1y1.avg_vis')
img1 = envi.open('/content/drive/MyDrive/Diploma/data_new/VNL_v2_npp_2016_global_vcmslcfg_c202102150000.average_masked_resampled.hdr', '/content/drive/MyDrive/Diploma/data_new/VNL_v2_npp_2016_global_vcmslcfg_c202102150000.average_masked_resampled')
x_train = img1.load()
y_train = img2.load()
clip_value = 2000
x_train = np.clip(x_train, 0, clip_value)
x_train /= x_train.max()
y_train /= y_train.max()

In [ ]:
def get_generator(x_train, y_train, quantile=0.001, patch_size=64, count=10000):
  
  mask = x_train > np.quantile(x_train, quantile)
  coords = np.array(np.where(mask)).T
  rng = np.random.default_rng()
  numbers = rng.choice(coords, size=count, replace=False)
  def generate_data_x():
    for c in numbers:
      patch = x_train[c[0] - patch_size: c[0], c[1] - patch_size: c[1]]
      if patch.shape == (patch_size, patch_size, 1):
        yield patch
  def generate_data_y():
    for c in numbers:
      patch =  y_train[c[0] - patch_size: c[0], c[1] - patch_size: c[1]]
      if patch.shape == (patch_size, patch_size, 1):
        yield patch
    
  return generate_data_x, generate_data_y

In [ ]:
# for i, j in zip(gen[0](), gen[1]()):
#   imshow(i)
#   imshow(j)

In [ ]:
def conv_block(n_filter, kernel_size,
               activation='relu',
               normalization=True,
               padding='same',
               dropout=0.0,
               num_blocks=2,
               residual=True
               ):
  
  def cnv(s):
    if normalization:
      s = tfa.layers.InstanceNormalization(
          axis=-1,
          center=True, 
          scale=True,
          beta_initializer="random_uniform",
          gamma_initializer="random_uniform"
          )(s)
    s = Activation(activation)(s)
    s = Conv2D(n_filter, kernel_size, padding=padding)(s)
    if dropout > 0.0:
      s = Dropout(dropout, )(s)
    return s

  def blocks(s):
    if residual:
      res = s
    for i in range(num_blocks):
      s = cnv(s)
    if residual:
      if res.shape[-1] != s.shape[-1]:
        res = Conv2D(s.shape[-1], (1,1))(res)
      s = tf.keras.layers.Add()([res, s])
    return s

  return blocks

In [ ]:
def unet(input_shape,
         last_activation=tf.keras.layers.ReLU(),
         n_depth=2,
         n_filter_base=16,
         kernel_size=(3,3),
         n_conv_per_depth=2,
         activation='relu',
         normalization=True,
         dropout=0.0,
         pool_size=(2,2),
         num_blocks=2
         ):

  input = Input(input_shape, dtype=tf.float32)
  layer = input

  skip_layers = []
  # print(layer.shape)
  # down
  for n in range(n_depth):
    for i in range(n_conv_per_depth):
      layer = conv_block(
          n_filter_base * 2 ** n,
          kernel_size,
          activation=activation,
          normalization=normalization,
          num_blocks=num_blocks,
          dropout=dropout,
          padding='same'
          )(layer)
    skip_layers.append(layer)
    layer = MaxPool2D(pool_size)(layer)

  # middle
  for i in range(n_conv_per_depth - 1):
    # print(layer.shape)
    layer = conv_block(
        n_filter_base * 2 ** n_depth,
        kernel_size, activation=activation,
        normalization=normalization,
        num_blocks=num_blocks,
        dropout=dropout,
        padding='same'
        )(layer)
  # print(layer.shape)
  layer = conv_block(
      n_filter_base * 2 ** max(n_depth - 1, 0),
      kernel_size,
      activation=activation,
      normalization=normalization,
      num_blocks=num_blocks,
      dropout=dropout,
      padding='same'
      )(layer)
  # print(layer.shape)
  # up
  for n in range(n_depth - 1, -1, -1):
    up = UpSampling2D(pool_size)(layer)
    layer = Concatenate()([up, skip_layers[n]])
    for i in range(n_conv_per_depth - 1):
      layer = conv_block(
        n_filter_base * 2 ** n,
        kernel_size,
        activation=activation,
        normalization=normalization,
        num_blocks=num_blocks,
        dropout=dropout,
        padding='same'
        )(layer)
    # print(layer.shape)
    layer = conv_block(
        n_filter_base * 2 ** max(n-1, 0),
        kernel_size,
        activation=activation,
        normalization=normalization,
        num_blocks=num_blocks,
        dropout=dropout,
        padding='same'
        )(layer)    
  output = conv_block(
    input_shape[-1],
    (1,) * len(kernel_size),
    activation='linear',
    normalization=normalization,
    num_blocks=1,
    padding='same',
    residual=False
    )(layer)
  # output = Add()([output, input])
  output = last_activation(output)

  return Model(inputs=input, outputs=output)

In [ ]:
def apply_preprocessing(x, y):
  # (x, y) = data
  x1 = tf.image.rot90(x)
  y1 = tf.image.rot90(y)
  x2 = tf.image.rot90(x1)
  y2 = tf.image.rot90(y1)
  x3 = tf.image.rot90(x2)
  y3 = tf.image.rot90(y2)
  x4 = tf.image.flip_left_right(x)
  y4 = tf.image.flip_left_right(y)
  x5 = tf.image.flip_up_down(x)
  y5 = tf.image.flip_up_down(y)
  return tf.data.Dataset.from_tensors((tf.convert_to_tensor((x,x1,x2,x3,x4,x5)), tf.convert_to_tensor((y, y1,y2,y3,y4,y5))))

In [ ]:
def train(    
    x_train,
    y_train,
    batch_size = 5,
    train_size = 100,
    epochs = 1,
    quant = 0.001,
    patch_size = 256,
    loss_func = tf.keras.losses.MeanAbsoluteError(),
    lr=0.002,
    lowest_lr=0.0002
    ):  


  metric = tf.keras.metrics.MeanAbsoluteError()

  @tf.function
  def step(x, y_true, optimizer):
      with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_func(y_true, y_pred)
      grads = tape.gradient(loss, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))
      metric.update_state(y_true, y_pred)
  
  # tf.profiler.experimental.start(log_dir)
  decay = tf.keras.optimizers.schedules.ExponentialDecay(
      lr,
      int(train_size * epochs * 6 / batch_size),
      lowest_lr / lr,
      staircase=False
      )
  optimizer = tf.optimizers.Adam(decay)
  for epoch in tqdm(range(epochs)):


    gen_x, gen_y = get_generator(x_train, y_train, quant, patch_size=patch_size, count=train_size)
    x_train_dataset = tf.data.Dataset.from_generator(gen_x, output_signature=tf.TensorSpec(shape=(patch_size, patch_size, 1), dtype=tf.float32))
    y_train_dataset = tf.data.Dataset.from_generator(gen_y, output_signature=tf.TensorSpec(shape=(patch_size, patch_size, 1), dtype=tf.float32))
    train_dataset = tf.data.Dataset.zip((x_train_dataset, y_train_dataset)).flat_map(apply_preprocessing).unbatch().shuffle(10000).batch(batch_size)
    for x, y_true in train_dataset:
      step(x, y_true, optimizer)
    # for x, y_true in test_dataset:
    #   y_pred = model(x)
    #   loss = tf.keras.losses.MeanAbsoluteError()(y_true, y_pred)
    print(' train_loss = {:.5f}, cur_lr = {:.5f}'.format(metric.result(), optimizer._decayed_lr(tf.float32).numpy()))
    metric.reset_states()
    # if epoch % 5 == 0:
    #   model.save_weights('/content/drive/MyDrive/Diploma/my_models/model_custom_loss_depth_3_balanced_ckpt_epoch{}'.format(epoch))

In [ ]:
%load_ext tensorboard
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_images=True, update_freq=100)

In [ ]:
model = unet((256, 256, 1), last_activation=tf.keras.layers.ReLU(1.), n_depth=4, n_filter_base=32, kernel_size=(3,3), n_conv_per_depth=3, dropout=0.2)
tensorboard_callback.set_model(model)
model.summary()
model.load_weights('/content/drive/MyDrive/Diploma/my_models/cur_model_epoch0.h5')
# model = tf.keras.models.load_model('/content/drive/MyDrive/Diploma/my_models/model_custom_loss.h5')

In [ ]:
def custom_loss(power=5/4):
  def my_loss(Y_true, Y_pred):
    return tf.abs(tf.reduce_mean(tf.pow(Y_true, power) - tf.pow(Y_pred, power), axis=-1))
  return my_loss

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
def perform_training(x_test, y_test, total=10, min_quantile=0.001, max_quantile=0.5, test_size=1024):
  patch_size = 256
  gen_x, gen_y = get_generator(x_test, y_test, 0.001, patch_size=patch_size, count=test_size)
  x_test_dataset = tf.data.Dataset.from_generator(gen_x, output_signature=tf.TensorSpec(shape=(patch_size, patch_size, 1), dtype=tf.float32))
  y_test_dataset = tf.data.Dataset.from_generator(gen_y, output_signature=tf.TensorSpec(shape=(patch_size, patch_size, 1), dtype=tf.float32))
  test_dataset = tf.data.Dataset.zip((x_test_dataset, y_test_dataset)).batch(24)
  metric = tf.keras.metrics.MeanAbsoluteError()
  # sup_model = tf.keras.models.clone_model(model)
  
  # print(sup_model.layers[3].get_weights())
  # for layer in sup_model.layers:
  #   w_and_b = layer.get_weights()
  #   w_and_b_new = []
  #   for w in w_and_b:
  #     w_and_b_new.append(np.zeros_like(w))
  #   layer.set_weights(w_and_b_new)
  # print(sup_model.layers[3].get_weights())
  quants = np.linspace(1, total, total)  
  quants = np.power(quants, 2)
  quants = quants / (total ** 2) * max_quantile
  print(quants)
  # return
  for epoch, q in enumerate(quants):
    train(
        x_train,
        y_train,
        epochs=10,
        train_size=1024,
        batch_size=16,
        patch_size=256,
        quant=q,
        # loss_func=custom_loss(),
        lr=0.0005,
        lowest_lr=0.0002
        )
    # if q == quants[0]:   
    #   sup_model.set_weights(model.get_weights())
    # else:
    #   for sup_layer, main_layer in zip(sup_model.layers, model.layers):
    #     w_and_b_sup = sup_layer.get_weights()
    #     w_and_b_sup_new = []
    #     w_and_b_main = main_layer.get_weights()
    #     for w_sup, w_main in zip(w_and_b_sup, w_and_b_main):
    #       new_w = (w_sup * (total - 1) + w_main ) / total
    #       w_and_b_sup_new.append(new_w)
    #     sup_layer.set_weights(w_and_b_sup_new)

    for x, y_true in test_dataset:
      y_pred = model(x)
      metric.update_state(y_true, y_pred)
    print('model MAE = {:.5f}'.format(metric.result()))
    metric.reset_states()
    model.save_weights('/content/drive/MyDrive/Diploma/my_models/cur_model_epoch{}.h5'.format(epoch))
  return model

In [ ]:
x_test = None
y_test = None


# img4 = envi.open('/content/drive/MyDrive/Diploma/data_new/F15_20150101_20151231.v4.stable_lights.shiftx1y1.avg_vis.hdr', '/content/drive/MyDrive/Diploma/data_new/F15_20150101_20151231.v4.stable_lights.shiftx1y1.avg_vis')
# img3 = envi.open('/content/drive/MyDrive/Diploma/data_new/VNL_v2_npp_2015_global_vcmslcfg_c202102150000.average_masked_resampled.hdr', '/content/drive/MyDrive/Diploma/data_new/VNL_v2_npp_2015_global_vcmslcfg_c202102150000.average_masked_resampled')
# x_test = img3.load()
# y_test = img4.load()
# clip_value = 2000
# x_test = np.clip(x_test, 0, clip_value)
# x_test /= x_test.max()
# y_test /= y_test.max()
# pred_num = 4


if x_test is None:
  x_test = x_train
  y_test = y_train
  pred_num = 3

In [ ]:
# model = perform_training(x_test, y_test, total=10, min_quantile=0.001, max_quantile=0.7, test_size=1024)

In [ ]:
# train(x_train, y_train, epochs=1, train_size=100, batch_size=12, patch_size=256, quant=0.002, loss_func=custom_loss())

In [ ]:
# patch_size = 256
# test_size = 5
# gen_x, gen_y = get_generator(x_train, y_train, 0.9, patch_size=patch_size, count=test_size)
# x_test_dataset = tf.data.Dataset.from_generator(gen_x, output_signature=tf.TensorSpec(shape=(patch_size, patch_size, 1), dtype=tf.float32))
# y_test_dataset = tf.data.Dataset.from_generator(gen_y, output_signature=tf.TensorSpec(shape=(patch_size, patch_size, 1), dtype=tf.float32))
# test_dataset = tf.data.Dataset.zip((x_test_dataset, y_test_dataset)).batch(1)
# for x, y_true in test_dataset:
#       y_pred = model(x)
#       imshow(x.numpy())
#       imshow(y_pred.numpy())
#       imshow(y_true.numpy())
#       loss = tf.reduce_mean(tf.keras.losses.mae(y_true, y_pred))
#       print(loss)

In [ ]:
model.compile()

In [ ]:
model.save('/content/drive/MyDrive/Diploma/my_models/model_MAE_depth_4_per_level_3.h5')

In [ ]:
# x_train = x_train[:-1, :-1]

In [ ]:
from skimage.util import view_as_windows, view_as_blocks

def gen_restore_data(x_train_windows):
  def gen():    
    for i in range(x_train_windows.shape[0]):
      for j in range(x_train_windows.shape[1]):
        cnt = np.count_nonzero(x_train_windows[i, j, 0] > 0)
        if cnt != 0:
          yield x_train_windows[i, j, 0]
  return gen

In [ ]:
from functools import wraps
from time import time
def measure(func):
    @wraps(func)
    def _time_it(*args, **kwargs):
        start = int(round(time() * 1000))
        try:
            return func(*args, **kwargs)
        finally:
            end_ = int(round(time() * 1000)) - start
            print(f"Total execution time: {end_ if end_ > 0 else 0} ms")
    return _time_it

In [ ]:
# measure
def get_window(a):
  a = np.squeeze(a)
  std = np.max(a.shape) / 4
  center = (np.array((a.shape[0], a.shape[1]))[np.newaxis, ...] - 1) / 2
  a = np.zeros_like(a)
  ind = np.array(np.where(a == 0))
  ind = np.moveaxis(ind, 0, -1).reshape((*a.shape, 2))
  a = np.linalg.norm(ind - center, ord=2, axis=-1)
  a = np.exp(-(np.power(a, 2) / (2 * std ** 2))) / (std * (2 * np.pi) ** (1 / 2))
  return a / a.max()


def perform_transfer(x, patch_size=256, step=64, cutoff=0):
  orig_shape=x.shape
  pad_values = (patch_size - 1 + step, patch_size - 1 + step)
  x = np.pad(x, ((0, pad_values[0]), (0, pad_values[1]), (0,0)))
  counter = np.zeros_like(x)
  y = np.zeros_like(x)
  y_coef = np.zeros_like(x)
  window_coef = np.full((patch_size, patch_size, 1), 10E-5)
  window_coef[..., 0] = get_window(window_coef[...])
  x_train_windows = view_as_windows(x, (patch_size, patch_size, 1), step=step)

  dataset = tf.data.Dataset.from_generator(
      gen_restore_data(x_train_windows),
      output_signature=tf.TensorSpec(shape=(patch_size, patch_size, 1)))
  dataset = dataset.batch(24)
  try:
    res = model.predict(dataset, verbose=0)
  except ValueError:
    return y[:orig_shape[0], :orig_shape[1]]
  counter = 0
  y_windows = view_as_windows(y, (patch_size, patch_size, 1), step=step)
  y_coef_windows = view_as_windows(y_coef, (patch_size, patch_size, 1), step=step)
  for i in range(y_windows.shape[0]):
    for j in range(y_windows.shape[1]):
      cnt = np.count_nonzero(x_train_windows[i, j, 0] > 0)
      if cnt != 0:
        y_windows[i, j, 0] += res[counter] * window_coef
        counter += 1
      y_coef_windows[i, j, 0] += window_coef
  # lol = np.ones_like(y)
  # lol[...] = 0
  y = y / (y_coef + 10E-5)
  return y[:orig_shape[0], :orig_shape[1]]

In [ ]:
from sys import getsizeof
def split_and_predict(x, split=(8, 8, 1), overlap=1/3, cutoff=5):

  step = np.array(x.shape) // np.array(split)
  overlap = step * overlap
  overlap = overlap.astype(int)
  step = step.astype(int)
  orig_shape=x.shape
  pad_values = (step[0] + overlap[0], step[1] + overlap[1])
  x = np.pad(x, ((cutoff, pad_values[0]), (cutoff, pad_values[1]), (0,0)))
  y = np.zeros_like(x)
  coefs = np.zeros_like(x)
  i_s_l = [i * step[0] for i in range(split[0] + 2)]
  # i_s_l.append(x.shape[0] - overlap)
  j_s_l = [i * step[1] for i in range(split[1] + 2)]
  # j_s_l.append(x.shape[1] - overlap)
  i_s_r = [i + overlap[0] for i in i_s_l[1:]]
  j_s_r = [i + overlap[1] for i in j_s_l[1:]]
  window_coef = get_window(np.zeros_like(x[:i_s_r[0], :j_s_r[0]]))[..., np.newaxis]
  window_aux = np.zeros_like(window_coef)
  window_aux[cutoff:-cutoff, cutoff:-cutoff] = 1
  np.multiply(window_coef, window_aux, out=window_coef)
  window_aux = None
  print(i_s_l, i_s_r, j_s_l, j_s_r)
  print(getsizeof(step), getsizeof(coefs),getsizeof(x),getsizeof(y), getsizeof(window_coef))
  # print(window_coef.shape)
  # return
  for i_l, i_r in tqdm(zip(i_s_l[:-1], i_s_r), total=len(i_s_r)):
    for j_l, j_r in zip(j_s_l[:-1], j_s_r):
      temp = perform_transfer(x[i_l: i_r, j_l: j_r])
      np.multiply(temp, window_coef, out=temp)
      np.add(y[i_l: i_r, j_l: j_r], temp, out=y[i_l: i_r, j_l: j_r])
      np.add(coefs[i_l: i_r, j_l: j_r], window_coef, coefs[i_l: i_r, j_l: j_r])
      # y[i_l: i_r, j_l: j_r] += perform_transfer(x[i_l: i_r, j_l: j_r]) * window_coef
      # coefs[i_l: i_r, j_l: j_r] += window_coef
      # print(i_l, i_r, j_l, j_r)
  np.add(coefs, 10E-5, out=coefs)
  np.divide(y, coefs, out=y)
  return y[cutoff:orig_shape[0] + cutoff, cutoff:orig_shape[1] + cutoff]

In [ ]:
x_train = y_train = y_test = None
res = split_and_predict(x_test)

In [ ]:
x_test.shape

In [ ]:
# x_train.shape

In [ ]:
res = res * 63
res = np.around(res, 0)

In [ ]:
res.max()

In [ ]:
y = 4983
x = 7621
imshow(res[y-256:y, x-256:x])

In [ ]:
y = 1000
x = 42000
imshow(res[y-210:y, x-300:x])

In [ ]:
np.savez('/content/drive/MyDrive/Diploma/Predictions/pred{}.npz'.format(pred_num), res[:,:,0])
from tifffile import imsave
# imsave('/content/drive/MyDrive/Diploma/preds_for_years/pred_{}.tiff'.format(2015), res[:,:,0])

In [ ]:
imshow(res)
# imshow(x_train[:2000, 40000:])